In [1]:
from torchvision.io import read_image
from torch.utils.data import Dataset, random_split, DataLoader
import os
from torch import nn
from torchsummary import summary
from torchvision import transforms


class CustomImageDataset(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir

    def __len__(self):
        _, _, files = next(os.walk(self.img_dir))
        file_count = len(files)
        return file_count

    def __getitem__(self, idx):
        _, _, files = next(os.walk(self.img_dir))
        img_path = files[idx]
        image = read_image(f'{self.img_dir}/{img_path}').float()
        transform = transforms.Compose([
            transforms.Resize((640, 640)),
        ])
        image = transform(image)
        return image

In [2]:
mask_dataset = CustomImageDataset('./masks')
print("length of the dataset is:", len(mask_dataset))

train, val, test = random_split(mask_dataset, [0.7, 0.2, 0.1])

print("length of the train dataset is:", len(train))
print("length of the val dataset is:", len(val))
print("length of the test dataset is:", len(test))


batch_size = 8

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

length of the dataset is: 66261
length of the train dataset is: 46383
length of the val dataset is: 13252
length of the test dataset is: 6626


In [3]:
from torch import flatten, reshape
import torch.nn.functional as F


class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 160, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(160, 80, 5)
        self.conv3 = nn.Conv2d(80, 40, 5)
        self.conv4 = nn.Conv2d(40, 20, 5)
        self.conv5 = nn.Conv2d(20, 10, 5)
        self.conv6 = nn.Conv2d(10, 5, 5)
        self.fc3 = nn.Linear(180, 64)
        self.upsample = nn.UpsamplingNearest2d(scale_factor=2)
        self.unf = nn.Linear(64, 180)
        self.unconv1 = nn.ConvTranspose2d(5, 10, 5)
        self.unconv2 = nn.ConvTranspose2d(10, 20, 5)
        self.unconv3 = nn.ConvTranspose2d(20, 40, 5)
        self.unconv4 = nn.ConvTranspose2d(40, 80, 5)
        self.unconv5 = nn.ConvTranspose2d(80, 160, 5)
        self.unconv6 = nn.ConvTranspose2d(160, 4, 5)
        self.finalUpsample = nn.UpsamplingNearest2d((636, 636))

    def encode(self, x):
        x = self.pool((self.conv1(x)))
        x = self.pool((self.conv2(x)))
        x = self.pool((self.conv3(x)))
        x = self.pool((self.conv4(x)))
        x = self.pool((self.conv5(x)))
        x = self.pool((self.conv6(x)))
        x = flatten(x, 1) # flatten all dimensions except batch
        x = self.fc3(x)
        return x

    def decode(self, x):
        x = self.unf(x)
        x = reshape(x, (-1, 5, 6, 6))
        x = self.unconv1(self.upsample(x))
        x = self.unconv2(self.upsample(x))
        x = self.unconv3(self.upsample(x))
        x = self.unconv4(self.upsample(x))
        x = self.unconv5(self.upsample(x))
        x = self.unconv6(self.finalUpsample(x))
        return x


    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x

In [4]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
decoder = AutoEncoder().to(device)
summary(decoder, (4,640, 640,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 160, 636, 636]          16,160
         MaxPool2d-2        [-1, 160, 318, 318]               0
            Conv2d-3         [-1, 80, 314, 314]         320,080
         MaxPool2d-4         [-1, 80, 157, 157]               0
            Conv2d-5         [-1, 40, 153, 153]          80,040
         MaxPool2d-6           [-1, 40, 76, 76]               0
            Conv2d-7           [-1, 20, 72, 72]          20,020
         MaxPool2d-8           [-1, 20, 36, 36]               0
            Conv2d-9           [-1, 10, 32, 32]           5,010
        MaxPool2d-10           [-1, 10, 16, 16]               0
           Conv2d-11            [-1, 5, 12, 12]           1,255
        MaxPool2d-12              [-1, 5, 6, 6]               0
           Linear-13                   [-1, 64]          11,584
           Linear-14                  [

In [5]:
import torch.optim as optim


criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(decoder.parameters(), lr=0.001)

AttributeError: 'Adam' object has no attribute 'to'

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = decoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

print('Finished Training')

/Users/mmandirola/tesis/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


: 